In [13]:
#Basic importations
import numpy as np
import tensorflow_datasets as tfds
import tensorflow_text as text
import tensorflow as tf


# This is mainly for deep understanding of paper Attention is all you need 
## Check it out here : https://arxiv.org/abs/1706.03762 .  
Basically is creating a transformer to be able to translate words from english to portuguese in this specific case.


In [14]:
#We use the data provided by Ted fundation in tensorflow_datasets
examples, metadata = tfds.load('ted_hrlr_translate/es_to_pt', with_info=True,
                               as_supervised=True)



train_examples, val_examples = examples['train'], examples['validation']

In [15]:
#we will call data a class to not have a random global variable 
class Data():
    def __init__(self):
        #alternitavely we can ask in innit what language we want to train
        i=0
        #WE WILL GET THE DATASET IN WORKING CONDITIONS EVERY LIST(TRAIN_EXMAPLES[I]) GIVES US A (EXAMPLE 1, EXAMPLE 2)
        pt=[]
        es=[]
        #WE NEED TO DECODE IT TO UTF-8 SO IT IS PRESENTABLE AND WE ONLY WANT THE NUMPY
        for es_example, pt_example in list(train_examples):
            pt.append((pt_example.numpy()).decode('utf-8'))
            es.append((es_example.numpy()).decode('utf-8'))
        self.pt=pt
        self.es=es
        #And with this we have or data


Now we need to think about input embedding .
Because obviously computers dont understand strings, we need some kind of input. We will build an embedding from zero
from tensorflow_text.tools.wordpiece_vocab import bert_vocab_from_dataset -> will help us generate a word for word construction
But for this case we need it to be object database so Data() is not useful in this case, although we will use it later


BertTokenizer is a type of embedding described in detail in https://arxiv.org/abs/1810.04805, we will not go in detail, we will assume its effectivness in embedding (as they describe is a bidirectional representation from unlabeled text )




In [16]:
from tensorflow_text.tools.wordpiece_vocab import bert_vocab_from_dataset as bert

In [17]:
#Datos=Data()
# in this case lets generate a vocabulary of 5000 words
# WE USE MAP AND LAMBDA MAP MAPS ONTO ARRAY AND SIMPLY LAMBDA ES,PT: ES IS SETTING FROM BOTH ES AND PT SETTING THE RESULT TO ES
# THE RESEVED TOKENS ARE SIMPLY START FINISH conditions (recall we need to start and finish the input and output)
es_vocab = bert.bert_vocab_from_dataset(train_examples.map(lambda es,pt:es),vocab_size =5000,reserved_tokens=["[PAD]", "[UNK]", "[START]", "[END]"])
pt_vocab = bert.bert_vocab_from_dataset(train_examples.map(lambda es,pt:pt),vocab_size =5000,reserved_tokens=["[PAD]", "[UNK]", "[START]", "[END]"])


KeyboardInterrupt: 

In [ ]:

def write_vocab_file(filepath, vocab):
    with open(filepath, 'w',encoding='utf-8') as f:
        for token in vocab:
              print(token, file=f)

In [ ]:
write_vocab_file('es_vocab.txt',es_vocab)
write_vocab_file('pt_vocab.txt',pt_vocab)



And with this we have already created a vocabulary. To  encapsulate everything we could put it in a class in this way.

In [8]:
class Vocabgen:
    def __init__(self,filename1,filename2,num):
        es_vocab = bert.bert_vocab_from_dataset(train_examples.map(lambda es,pt:es),vocab_size =num,reserved_tokens=["[PAD]", "[UNK]", "[START]", "[END]"])
        pt_vocab = bert.bert_vocab_from_dataset(train_examples.map(lambda es,pt:pt),vocab_size =num,reserved_tokens=["[PAD]", "[UNK]", "[START]", "[END]"])
        #WE CREATE THE VOCABS AND WE WRITE THEM
        write_vocab_file(filename1,es_vocab)
        write_vocab_file(filename2,pt_vocab)
        #We could make it even better so that given two languages we create the vocabs but is not necessary at the moment
        
    def write_vocab_file(filepath, vocab):
        with open(filepath, 'w',encoding='utf-8') as f:
            for token in vocab:
                  print(token, file=f)

In [11]:
Vocabgen('es_vocab.txt','pt_vocab.txt',5000)

Now we can easily create the tokenizers using tensorflow_text.BertTokenizer(atext, **kargs)

In [18]:
class Tokenizer:
    def __init__(self):
        #the tokenizer class will have tokens of class BertTokenizer as .pr .es 
        self.pt = text.BertTokenizer('pt_vocab.txt', **{})
        self.es =text.BertTokenizer('es_vocab.txt', **{})

def add_start_end(ragged): #this function has been copied from tensorflow  guide to subwords
        #will add the start and end
        
        reserved_tokens=["[PAD]", "[UNK]", "[START]", "[END]"]
        START = tf.argmax(tf.constant(reserved_tokens) == "[START]")
        END = tf.argmax(tf.constant(reserved_tokens) == "[END]")
        count = ragged.bounding_shape()[0]
        starts = tf.fill([count,1], START)
        ends = tf.fill([count,1], END)
        return tf.concat([starts, ragged, ends], axis=1)
        
#AND WE CREATE ANOTHER CLASS THAT CONTAINS OUR TOKENIZER TO TOKENIZE AND DE-TOKENIZE
class Embedding:
    Tokens=Tokenizer()
        #tokenize ---->
        #This returns an output of (batch,word,wordpiece)
        #What we mean is that we have -> I am good. output-> [ [ [I(values)] , [am(values)] , [good(values)]] ]
    
    def detokenize(self,detokenizedata):
        detokenizept=(Tokens.pt.detokenize(detokenizedata[1]))
        detokenizees=(Tokens.es.detokenize(detokenizedata[0]))
        
        detokenizees= tf.strings.reduce_join(detokenizees, separator=' ', axis=-1)
        detokenizept= tf.strings.reduce_join(detokenizept, separator=' ', axis=-1)
        merged=[]
        merged.append(detokenizees)
        merged.append(detokenizept)
        return merged
    def tokenize(self,tokenizedata):
        tokenizept=(Tokens.pt.tokenize(tokenizedata[1])).merge_dims(1,2)
        tokenizees=(Tokens.es.tokenize(tokenizedata[0])).merge_dims(1,2)
        #this part is for preperation for our purposes
        tokenizept=(add_start_end(tokenizept)).to_tensor()# we apply to tensor to make the output a normal tensor
        tokenizees=(add_start_end(tokenizees)).to_tensor()# we apply to tensor to make the output a normal tensor
        
        #THIS TO TENSOR WILL MAKE THAT ON TH OUTPUT WE GET A LOT OF PADDING but thats easily removable
        
        merged=[]
        merged.append(tokenizees)
        merged.append(tokenizept)
        return merged
    
    
    
    
    

In [19]:
Datos=Data()
Embed=Embedding()

To show the conversions lets do a simple example

In [21]:
test=[]
testes=[]
testpt=[]
testpt.append(Datos.pt[12])#WE INCLUDE TWO EXAMPLES SO WE CHECK VARIABILITY OF BATCH SIZE
testpt.append(Datos.pt[13])
testes.append(Datos.es[13])
testes.append(Datos.es[12])
print(Datos.pt[12])
test.append(testes)
test.append(testpt)
tokenizeddata=Embed.tokenize(test)
#We will try to detokenize to see if its working!
print(tokenizeddata)

print((Embed.detokenize(tokenizeddata)[1].numpy())[1].decode('utf-8')) #this goes to the pt detokenize data and decodes 
#it in a tensor
#then we are specifically getting the first text and decoding it  
print((Datos.pt[13]))
#ITS REALLY IMPORTAnt to decode de output, it will be coded in utf-8 to make it leggible we need to decode 
#AND as it can be seen we have a desired output only some random spaces that we do not need to bother about as differences
#IT IS ALSO INCLUDED THE START AND END WHICH IS EXACTLY WHAT WE NEED



`` eles não dizem : `` '' quero água quente nos chuveiros . ''
[<tf.Tensor: shape=(2, 23), dtype=int64, numpy=
array([[   2,  282,   16,   10,  281,   11,    3,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0],
       [   2,   38,   38,  197,  776,   14,   38,   38,    9,    9,  324,
         333,  442, 2805,  191,  200,   42, 1070, 2101,   16,    9,    9,
           3]], dtype=int64)>, <tf.Tensor: shape=(2, 24), dtype=int64, numpy=
array([[   2,   39,   39,  162,  117,  658,   28,   39,   39,    9,    9,
         264,  373, 1950,  133,   42,  523,  201,  557, 1674,   16,    9,
           9,    3],
       [   2,  282,   16,   10,  209,   11,    3,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0]], dtype=int64)>]
[START] obrigado . ( aplausos ) [END] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]
obriga

## Usage of class Embed
Embed has Embed.tokenizedata(data) ->(tokenizedspanish,tokenizedpt)as dtype= Tensor


Embed has Embed.detokenizedata(data) ->(detokenizedspanish,detokenizedpt) as dtype= Tensor


Be careful with detokenized as it should be decoded again!
For now they are only crated for the specific case of spanish and pt.

It will include de START AND END ALSO in both 

## Usage of class Data
Data() -> Data.pt ->(Data pt)


       -> Data.es ->(Data es) 


# ALL IS HEAVILY INFLUENCED BY https://www.tensorflow.org/text/guide/subwords_tokenizer?hl=es-419%C3%A7

As it is just a learning project

In [12]:
BATCH_SIZE=2
BUFFER_SIZE=20000
Embed=Embedding()
Tokens=Tokenizer()
def tokenize_pairs(es,pt):
    data2=[]
    data2.append(es)
    data2.append(pt)
    pt = Embed.tokenize(data2)[1]
    # Convert from ragged to dense, padding with zeros.
    es=Embed.tokenize(data2)[0]
    # Convert from ragged to dense, padding with zeros.
    return es,pt
def make_batches(ds):
    return (
      ds
      .cache()
      .shuffle(BUFFER_SIZE)
      .batch(BATCH_SIZE)
      .map(tokenize_pairs, num_parallel_calls=tf.data.AUTOTUNE)
      .prefetch(tf.data.AUTOTUNE))

examples, metadata = tfds.load('ted_hrlr_translate/pt_to_en', with_info=True,
                               as_supervised=True)


train_examples, val_examples = examples['train'], examples['validation']
train_batches=make_batches(train_examples)
for (batch, (inp, tar)) in enumerate(train_batches):
    print(inp)


tf.Tensor(
[[   2 1846   14   52 4816  274  564  480  188   59  501   51  306  441
  1659   42  274   58  306  424  736  653  413   57  653  274 1010   16
     3    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0]
 [   2   43 1446 3406  496   57 1400  301 2124  188 3087  852 1767  189
    39  198    1  189 1351  424 2017  501   59  501  801 2439  336   41
   653 1870  234  605 1107   15   20   42 1070  786 1797  501   53  396
   501 3084  188  265  605 1008  440   54  336 2269  274  258  356   16
     3]], shape=(2, 57), dtype=int64)
tf.Tensor(
[[   2   53  852 3187 2155   16    3    0    0    0    0    0    0    0
     0    0]
 [   2  191  413 4816  274   14  208   44  244  577  525   47  198  502
    32    3]], shape=(2, 16), dtype=int64)
tf.Tensor(
[[   2   53   39 3091 1008  231  208   53   40 3791  446   16    3    0
     0]
 [   2   84   41 2547  786  710 3161  227  229   39   58 4820  50

tf.Tensor(
[[   2   84   59  501   46 4135  501  197  413  369  878  292   16    3
     0    0    0    0    0    0    0    0    0    0    0    0]
 [   2   38   38   46  786   28   38   38    9    9   52 4816  274   47
  4357  198   39  215  212  653  244  682   16    9    9    3]], shape=(2, 26), dtype=int64)
tf.Tensor(
[[   2   43   14  188   44 3931   14  489   46  369  468 2396  198    3
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0]
 [   2   52 4816  274   84 1771   39  396  227  358  189   53   45  676
    54 2406   43   53 2978   54 2676   14 4252   58 2868  852  352  501
    53  198 3364  198   53 2978   53 2279   84   54  582  475 1070  577
   869   43   58  935  198 3084 1509  234   16    3]], shape=(2, 52), dtype=int64)
tf.Tensor(
[[   2   38   38   38   38    9    9  188  424  886  244   15  210  965
    15  396   59  501   39 

tf.Tensor(
[[   2  203  789  328   14  199  391   44  244  577  756   54  582  736
   444  926   52 4337  189 3820 4816  274   14  396  238  188  396  227
    51 3992  613   53  189   54  710  991 4543   16    3]
 [   2  242   41 4135 4818  369  238  208   54 3187  212  715 3508  415
   638   16    3    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0]], shape=(2, 39), dtype=int64)
tf.Tensor(
[[   2   51 3195  653  756  241  189   54  509  526  501  396  227   43
   886  920  502  188 1932  135   59  463  257   51  244  665   14  189
   227   53  396  212  653 2868   53 1070   52 4816  274  135   58 4820
   501   58  244  886  418   51  244  665 3838  188   39  852 3653 2987
    16    3]
 [   2   54  389  502   59  198  369   15  190  202   59  463   54  756
   518   14   40  358 4818  274   14   55 1307 1008  227   41  274 2406
    42  274 3023   16    3    0    0    0    0    0    0    0    0    0
     0    0    0    0    0 

tf.Tensor(
[[   2  192  241  727  501   53  189  188  878  501   44  244  577  756
    14 4252   52 4816  274   53  193  413 4816  274   39   44  244  577
   756   16    3    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0]
 [   2  211  736 1070  212  198  188   52  441  198  396  238   59  501
   367  188  211  435  356   39  212 2340   15   56  274 1070  810 2124
   422   14   43  211  736 1070  212  198  188   52  441  198   52 4816
   274  396  501   16    3]], shape=(2, 47), dtype=int64)
tf.Tensor(
[[   2 4252  202   47  198  502   14   60  274 1070   54  710  715 2406
   227  188   39 4801 2416   14   43   39  210  292  653 1400   54  241
   502  244  202  210   39 4801 2416  227   39   39  786  444 3380   53
  1130   84   53   54 3187  356 2331   46 4135  501  189   53  199 2269
  1089 1070   14   53 3317   16   41  358  424  736  479 1116   16    3]
 [   2   54  509  288  501  197  193  884 4818  274   53 2978   54  418
   526  238   39   51  244 

tf.Tensor(
[[   2  312   39  214  244   41  653  336 2155   39  274  190 3156  244
   616  690   39 1424 2161  188  235   53   51  244  665   48  274  878
   501   14  981 1070  188   47  356 2015  328  189  270 3407   59  501
   359 2124  396  227  238  188   44 1046  198  424 2017 4818 4816  274
   188 1919   16    3]
 [   2   54  274  664  501 1350   47 4603  203 1689  535   56 1070 4374
   198   14   54  418 2161  369   39  607  429 2124   42  274  665  635
    16    3    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0]], shape=(2, 60), dtype=int64)
tf.Tensor(
[[   2   43   39  227 1302 1115   41  274 1302  935  336   14  208 1211
    42  274  766   14   44  274  424   59  501  280   54 3187  356 2331
   198   39 1590  202   50  424  501 1947   39  204 1107  983 4816  274
    16    3]
 [   2   47 4603   84  208 2253  369  199 1590  336  501  197  209  301
    16    3    0    0    

...

tf.Tensor(
[[   2   53  198   44 2210  444   51  244  665  865   52  244  773 2447
    57 4816  274   53  198  746   43   39  198    1   16    3]
 [   2   60  274 1070 1318   15   60  389   42 2025  198  205  418   16
     3    0    0    0    0    0    0    0    0    0    0    0]], shape=(2, 26), dtype=int64)
tf.Tensor(
[[   2  209  501   16   43 1070  219 1009   39  188 3625 3805   39  198
    44  612  301 2969  198   42  274 2568   16    3]
 [   2   39  462 1581  424   53  199  715 1070 2977  369  690   16    3
     0    0    0    0    0    0    0    0    0    0]], shape=(2, 24), dtype=int64)
tf.Tensor(
[[   2 1526   39  627   14  391   41  653 4113   51  244  665   54  756
   328    3    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0]
 [  

tf.Tensor(
[[   2  188   44 3931   14  396  212  653  274  211  736 1070  212  198
    43  886  336  501  786  411   14  229  189  210  292  653 1400  981
  1364  466 4818  244   42  274  189 1318   39  214  244   47 1582 2340
   640   32    3    0    0    0    0    0    0    0    0    0    0    0]
 [   2   38   38   46  274  785   14   53 1070 4410   54  241  502  418
    39  927  577  756   41  274 2406  198  208   28   38   38    9    9
   396  238   59  501  359   52  241  301  460   14   43   41  653 1870
    15  199    9   51 1070 4818 3195 1324  389    9   16    9    9    3]], shape=(2, 56), dtype=int64)
tf.Tensor(
[[   2   58  227 1070 3815  188 2193   14   54  509  502  189   57 4816
   274    1   14  339   56 1709 2543   28  199  501   54  582  301 4818
  4816  274   16    3    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    

tf.Tensor(
[[   2   47  198  502   39  595  446  640 1070   14 2124  481  305   14
   229   52 4816  274  270 3407   52  509  653 2797   16    3]
 [   2  385 1364 1070  352  501  228   53 1070  878   44 1046  435   16
     3    0    0    0    0    0    0    0    0    0    0    0]], shape=(2, 26), dtype=int64)
tf.Tensor(
[[   2  219   44  274  424   39   54 3187  356 2969  701  501 3972  653
   244  605   56  336  595  664  135   39  215  212  653  244   53  227
  1197 1304 4818 4816  274  135    3    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0]
 [   2   38   38   54  509  502  189   55 1307 1008  227   59  501  189
  3491   52 2796  898   59  227 3933  225  199  501  786  710   39  504
    38   38    9    9   45  358 3626 2019  424    9    9    9    9   14
    43   46  369   15   53  198  188  242   53  198 1147   16    9    9
     3]], shape=(2, 57), dtype=int64)
tf.Tensor(
[[   2   43   47  198  502  396  878   59  50

tf.Tensor(
[[   2  416  878  577 1789  198  288  235   59  501  193  715 3187  616
   690  191 4802 2025 3808   52 4816  274  270 3407   59  463  685   39
   616 1463   14   53 1070   39  815   39   54  274 2015  501  503  208
   800  188 3368   14   53 1070   14  208   51 3992  198 1802  188 3087
   852 1767 2868   14   59  501  319   51 3992   14   51 3992  780  202
   211  736 2797 2760  336   41 2547   39   52 2868 1400 1186   16    3]
 [   2  203  789  328   39  863  189   84   40 2547   16    3    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0]], shape=(2, 84), dtype=int64)
tf.Tensor(
[[   2  211  736 2797  198  605  198   41  274 2406  198  189  262  621
    57 481

KeyboardInterrupt: 